In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf


## Question 1 & 2

Given data about an instrumental variable, find the effect. X is the treatment, W the confounder, Y the outcome, and Z the instrument. Use homework_4.1.csv. 

In [4]:
df_4_1 = pd.read_csv('homework_4.1.csv')
#df_4_1.head()

Subtract the average ﻿Y﻿ value for ﻿Z equals 1﻿ and ﻿Z equals 0﻿. Subtract the average ﻿X﻿ value for ﻿Z equals 1﻿ and ﻿Z equals 0﻿. Divide the two. 

In [5]:
avg_y_z1 = df_4_1[df_4_1['Z'] == 1]['Y'].mean()
avg_y_z0 = df_4_1[df_4_1['Z'] == 0]['Y'].mean()
avg_x_z1 = df_4_1[df_4_1['Z'] == 1]['X'].mean()
avg_x_z0 = df_4_1[df_4_1['Z'] == 0]['X'].mean()

# Apply formula
result = (avg_y_z1 - avg_y_z0) / (avg_x_z1 - avg_x_z0)
print("Result:", result)

Result: 1.5618587073955674


Find the average ﻿Y﻿ value for ﻿Z equals 1﻿ and ﻿Z equals 0﻿ for a narrow range of ﻿W﻿. Find the average ﻿X﻿ value for ﻿Z equals 1﻿ and ﻿Z equals 0﻿ for the same narrow range of ﻿W﻿. Take the ratio to find the effect. Then average this over all the ranges of ﻿W﻿. 


In [8]:
data_sorted_4_4 = df_4_1.sort_values("W")
num_bins = 5  # You can adjust the granularity
data_sorted_4_4["W_bin"] = pd.qcut(data_sorted_4_4["W"], q=num_bins, duplicates='drop')

# Container for local effects
local_effects_4_1 = []

# Loop over each bin
for bin_label, group in data_sorted_4_4.groupby("W_bin", observed=True):
    z1 = group[group["Z"] == 1]
    z0 = group[group["Z"] == 0]

    # Only proceed if both Z=1 and Z=0 are present in the bin
    if len(z1) > 0 and len(z0) > 0:
        avg_y_diff = z1["Y"].mean() - z0["Y"].mean()
        avg_x_diff = z1["X"].mean() - z0["X"].mean()

        # Avoid divide by zero
        if avg_x_diff != 0:
            local_effects_4_1.append(avg_y_diff / avg_x_diff)

# Final estimate: average of local effects
final_effect_4_1 = np.mean(local_effects_4_1)
print("Estimated Effect (Average over W bins):", final_effect_4_1)

Estimated Effect (Average over W bins): 1.5156437668382101


## Question 3-5

Given student data involving test scores (﻿X﻿), a cutoff, and an outcome (Y), which measures whether the students got into college (as in the example in the text), determine whether the math course helps students get into college in each dataset. Use datasets homework_4.2.a and homework_4.2.b. 

In [9]:
df_4_2 = pd.read_csv('homework_4.2.a.csv')
df_4_2 = pd.read_csv('homework_4.2.b.csv')